In [ ]:
# names for available stat blocks from .txt file read as string
inFile = open("5e_Data/monster_names_5e.txt")
name_stats = inFile.read().replace(" ", "").replace("-", " ").title()
inFile.close()
names = name_stats.split(",")

In [ ]:
# tkinter imports
import tkinter as tk
from tkinter import Frame
from tkinter import NONE
from tkinter import END
from tkinter import Label
from tkinter import Toplevel
from tkinter import Canvas
from tkinter import Entry
from tkinter import ttk
from tkinter import Scrollbar

In [ ]:
# web scraping package imports
import requests
import urllib
from urllib.error import HTTPError
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
# image file package imports
from io import BytesIO
from PIL import ImageTk
from PIL import Image

In [ ]:
# initialize counter and dictionaries
j = 0
enc_images = {}
stat_images = {}

# function for scraping webpages for image files
def getImage(thetext):
    global enc_image
    inp = thetext.replace(" ", "-")
    URL = "https://www.dndbeyond.com/monsters/{}".format(inp)
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
    req = urllib.request.Request(URL, headers=hdr)
    html = urlopen(req)
    bs = BeautifulSoup(html, 'html.parser')
    images = bs.find_all('img', src=True)
    
    for i in range(0, len(images)):
        try:
            img = images[i]['class']
            if images[i]['class'][0] == "monster-image":
                source = images[i]['src']
                if source.endswith("/humanoid.jpg"):
                    enc_image = Image.open("5e_Pics/humanoid.jpg")
                    break
                if source.endswith("/beast.jpg"):
                    enc_image = Image.open("5e_Pics/beast.jpg")
                    break
                else:
                    img_URL = images[i]['src']
                    response = requests.get(img_URL)  
                    enc_image = Image.open(BytesIO(response.content))
                    break
            else:
                continue
        except KeyError:
            pass

# function for retrieving .png files from local folder
def statBlocks(inp):
    global stat_image
    st_image = "5e_Stats/{}.png".format(inp)
    stat_image = Image.open(st_image)

# main application
class application(tk.Tk):
    def __init__(self, *args, **kwargs):
        global enc_start_image
        tk.Tk.__init__(self, *args, **kwargs)
        tk.Tk.iconbitmap(self, default="5e_Pics/d&d_icon.ico")
        tk.Tk.wm_title(self, "5e Monster Search")
        start_image = Image.open("5e_Pics/dragon_100.png")
        enc_start_image = ImageTk.PhotoImage(start_image)
        container = tk.Frame(self)
        container.pack(fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        self.frames = {}
        frame = StartPage(container, self)
        self.frames[StartPage] = frame
        frame.grid(row=0, column=0)
        self.show_frame(StartPage)
        self.attributes("-topmost", True)
        
    # raise frame of choice to front of container  
    def show_frame(self, controller):
        frame = self.frames[controller]
        frame.tkraise()
        
# first page to be shown        
class StartPage(tk.Frame):
    def __init__(self, parent, controller, *args, **kwargs):
        global variable
        tk.Frame.__init__(self, parent)
        self.pack()
        label = ttk.Label(self, text="Search Monsters:", font='Helvetica 12 bold')
        label.grid(row=1, column=1, padx=10, pady=10)
        self.entry = ttk.Entry(self)
        self.entry.grid(row=1, column=2, padx=10, pady=10)
        self.entry.bind("<Return>", self.validateAndNewWindow)
        self.entry.focus_set()
        label_im = ttk.Label(self, image=enc_start_image)
        label_im.grid(row=2, column=1, padx=10, pady=10, rowspan=2, sticky='nsew')
        self.dropdownlist = ttk.Combobox(self, values=names)
        self.dropdownlist.grid(row=3, column=2, padx=10, pady=10)
        self.dropdownlist.bind("<<ComboboxSelected>>", self.validateAndNewWindow)
        submitbutton = ttk.Button(self, text="Search", command=self.validateAndNewWindow)
        submitbutton.grid(row=2, column=2, padx=10, pady=10)
        resultlabel = ttk.Label(self, text="Last Entry:", font='Helvetica 12 bold')
        resultlabel.grid(row=4, column=1, padx=10, pady=10)
        self.resultbox = ttk.Entry(self)
        self.resultbox.grid(row=4, column=2, padx=10, pady=10)
    
    # validate entry box input and call on new window class
    def validateAndNewWindow(self, event=None):
        global thetext
        # if entry box is empty use drop down input
        if self.entry.get() == "":
            thetext = self.dropdownlist.get()
        else:
            thetext = self.entry.get()
          
        # if input is in names, create popup
        if thetext.title() in names:
            self.resultbox.delete(0, END)
            self.resultbox.insert(0, thetext.title())
            self.entry.delete(0, END)
            try:
                getImage(thetext.lower())
            except HTTPError:
                enc_image = Image.open("5e_Pics/beast.jpg")
            pop = popUp()
        else:
            result = "Monster Not Found!"
            self.entry.delete(0, END)
            self.resultbox.delete(0, END)
            self.resultbox.insert(0, result)

# class for creating multiple new windows
class popUp(tk.Tk):
    def __init__(self, *args, **kwargs):
        global j, enc_stat_image, enc_image
        
        # function to set scrollregion based on size of canvas after being packed with widgets
        def on_configure(event):
            canvas.configure(scrollregion=canvas.bbox('all'))
            
        self.newwindow = tk.Toplevel()
        self.newwindow.wm_title("{} Stat Block".format(thetext.title()))
        
        # "Image Coming Soon" if image not not retrieved from web
        try:
            enc_images[j] = ImageTk.PhotoImage(enc_image)
        except ValueError:
            enc_image = Image.open("5e_Pics/image-coming-soon.jpg")
        
        # fit canvas width to image width for images larger than 400 pixels wide
        if enc_image.size[0] > 400:
            canvas = tk.Canvas(self.newwindow, width=enc_image.size[0], height=862)
        else:
            canvas = tk.Canvas(self.newwindow, width=400, height=862)
            
        canvas.pack(side='left', expand=True, fill='both')
        vbar = ttk.Scrollbar(self.newwindow, orient='vertical', command=canvas.yview)
        vbar.pack(side='right', fill='y')
        canvas.config(yscrollcommand=vbar.set)
        canvas.bind('<Configure>', on_configure)
        frame = tk.Frame(canvas)
        canvas.create_window((0, 0), window=frame, anchor='nw')
        label_nm = ttk.Label(frame, text=thetext.title(), font='Helvetica 16 bold')
        label_nm.pack(side='top')
        enc_images[j] = ImageTk.PhotoImage(enc_image)
        enc_image.close()  
        label_im = ttk.Label(frame, image=enc_images[j])
        label_im.pack(side='top')
        
         # "Image Coming Soon" if stat block image file doesnt exist
        try:
            statBlocks(thetext.lower())
            stat_images[j] = ImageTk.PhotoImage(stat_image)
            stat_image.close()
        except FileNotFoundError:
            stock_image = Image.open("5e_Pics/image-coming-soon.jpg")
            stat_images[j] = ImageTk.PhotoImage(stock_image)
            stock_image.close()
        
        label_st = ttk.Label(frame, image=stat_images[j])
        label_st.pack(side='top')
        quitbutton = ttk.Button(frame, text='Close Window', command=self.close_windows)
        quitbutton.pack(side='top')
        j += 1        

    # function to destroy current window
    def close_windows(self):
        self.newwindow.destroy()

# main function to call on main application class and initialize tkinter
def main():
    app = application()
    app.mainloop() 

# call on main if module is main program
if __name__ == '__main__':        
    main()